<a href="https://colab.research.google.com/github/pasquale90/mthesis/blob/master/code/1.raw_ESC-50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#prevent from disconnecting --> to console
'''
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}setInterval(ClickConnect,60000)
'''

In [ ]:
#Import Google_drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#Import libraries
import os
import pandas as pd
import librosa
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim 
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm_notebook as tqdm
import math
from torch.autograd import Variable
from torchvision import transforms, models
from google.colab import files as filez

In [ ]:
#define device
if torch.cuda.is_available():
  device=torch.device('cuda:0')
else:
  device=torch.device('cpu')
print(device)

cuda:0


In [ ]:
#define experiment mode
class mode_class:
  def __init__(self, mode_atr):
    if (mode_atr == 8):
      self.mode='8'
    elif (mode_atr == 16):
      self.mode='16'
    elif (mode_atr == 22):
      self.mode = '22'
    elif (mode_atr == 32):
      self.mode = '32'
    else:
      print(f'{mode_atr} input attribute is not valid.Please insert 8 or 16 or 22 or 32')
  def get_mode(self):
    return self.mode

mode_id = 8
mode_id = 16
mode_id = 22
mode_id = 32

#define mode
mode_instance = mode_class(mode_id)
mode=mode_instance.get_mode()
print('mode:',mode)

mode: 32


In [ ]:
#define expid
expid='esc1'

#define paths
data_path='/content/gdrive/My Drive/dissertation/ESC-50-master/meta/esc50.csv'
audio_path='/content/gdrive/My Drive/dissertation/ESC-50-master/audio/'

#model	
model_savepath= "/content/"+expid+mode+"/saved_models/"

#store results
results_path="/content/"+expid+mode+"/results/"
compare_results_path ="/content/"+expid+mode+"/k_fold_results/"	

#attempt.py
attempt_path="/content/"+expid+mode+"/expattempt/"

In [ ]:
#Import Dataset
esc50 = pd.read_csv(data_path)
audiofiles = os.listdir(audio_path)
print(esc50.shape)
print(len(audiofiles))

#store_sorted_class_names, in the same way that are returned from dataset_class in data.py
esc_classes = sorted(esc50['category'].unique())
num_classes = len(esc_classes)
print('num_classes: ',num_classes)

folds = sorted(esc50['fold'].unique())
print('folds: ',num_classes)

(2000, 7)
2000
num_classes:  50
folds:  50


In [ ]:
#AUDIO AUGMENTATION FUNCTIONS_ synthetic data
def audio_augmentation(data, sr, class_conditional, shift_time=True, thresshold=0.5):

  #add_white_noise to the signal
  def white_noise(data): 
    noiz = np.random.randn(len(data))
    mean_intensity = np.sum(np.square(data))/len(data)
    data_wn = data + noiz*0.75* mean_intensity
    return data_wn
    
  #Shift the sound wave by a factor value chosen randomly within [0.5,1,1.5,2] seconds
  def time_shift(data,sr):
    time_step = np.random.choice([sr//2,sr,sr+sr//2,sr*2])
    time_shifted = np.roll(data,time_step)
    return time_shifted

  #Time-stretching the wave by a factor value of 0.9. Permissible : 0 < x < 1.0
  def time_stretch(data):#,factor
    factor = 0.90
    time_streched = librosa.effects.time_stretch(data,factor)
    return time_streched[0:len(data)]

  #pitch shifting of wave by a random factor value in the space [-1,1].  Permissible : -5 <= x <= 5
  def pitch_shift(data,sr):
    detune = np.random.uniform(low=-2.5,high=-1.75,size=None)
    overtune = np.random.uniform(low=1.75,high=2.5,size=None)
    shift_factor = np.random.choice([detune,overtune])
    pitch_shifted = librosa.effects.pitch_shift(data,sr,n_steps=shift_factor)
    return pitch_shifted
  
  def soft_pitch_shift(data,sr):
    detune = np.random.uniform(low=-1.0,high=-0.5,size=None)
    overtune = np.random.uniform(low=0.5,high=1.0,size=None)
    shift_factor = np.random.choice([detune,overtune])
    pitch_shifted = librosa.effects.pitch_shift(data,sr,n_steps=shift_factor)
    return pitch_shifted

  '''
  #A rough but very simple segmentation mask function
  def envelope(y,rate,threshold):
    mask = []
    y= pd.Series(y).apply(np.abs)
    y_mean = y.rolling(window=int(rate/10), min_periods=1, center=True).mean()#/10 means that win=1/10 sec
    for mean in y_mean:
      if mean > threshold:#if above the threshold keep
        mask.append(True)
      else:
        mask.append(False)#else drop
    return mask
  '''
  
  strong_augs = ['airplane','car_horn','cat',#esc
                 'chirping_birds','church_bells',
                 'cow','crow','crying_baby',
                 'door_wood_creaks','insects',
                 'rooster','sheep','siren',
                 'car_horn','children_playing','siren']#us8k

  medium_augs= ['breathing','brushing_teeth',#esc
                'clock_alarm','coughing','dog',
                'door_wood_knock','fireworks',
                'frog','glass_breaking','hand_saw',
                'hen','laughing','pig','pouring_water',
                'sneezing','snoring',
                'dog_bark','gun_shot','street_music']#us8k
                
  weak_augs = ['can_opening','chainsaw','clapping',#esc
               'clock_tick','crackling_fire','crickets',
               'drinking_sipping','engine','footsteps',
               'helicopter','keyboard_typing','mouse_click',
               'rain','sea_waves','thunderstorm',
              'toilet_flush','train','vacuum_cleaner',
              'washing_machine','water_drops','wind',  
               'air_conditioner','drilling','engine_idling','jackhammer']#us8k
   
  #prob_wn = np.random.uniform(low=0,high=1)
  #if prob_wn>thresshold:
  data =  white_noise(data)
  
  #prob_tsh = np.random.uniform(low=0,high=1)
  #if prob_tsh>thresshold:

  # if it is padded , don 't shift
  if shift_time: 
    data = time_shift(data,sr)

  if class_conditional in strong_augs:
    
    prob_tst = np.random.uniform(low=0,high=1)
    if prob_tst>thresshold:
      data = time_stretch(data)

    prob_psh = np.random.uniform(low=0,high=1)
    if prob_psh>thresshold:
      data = pitch_shift(data,sr)
  
  elif class_conditional in medium_augs:
    
    prob_spsh = np.random.uniform(low=0,high=1)
    if prob_spsh>thresshold:
      data = soft_pitch_shift(data,sr)

  elif class_conditional in weak_augs:
    pass

  return data

#Νο Vision Augmentations in this exp
'''
train_transforms = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
        transforms.RandomErasing(p=0.5, scale = (0.05,0.05), ratio = (0.3,0.33), value=0, inplace=False)      
    ])
valid_transforms = transforms.Compose([
        transforms.ToTensor()
    ])
'''

'\ntrain_transforms = transforms.Compose([\n        transforms.RandomHorizontalFlip(),\n        transforms.ToTensor()\n        transforms.RandomErasing(p=0.5, scale = (0.05,0.05), ratio = (0.3,0.33), value=0, inplace=False)      \n    ])\nvalid_transforms = transforms.Compose([\n        transforms.ToTensor()\n    ])\n'

In [ ]:
#define analysis parameters
def analysis_parameters(mode):
    sampling_rate=int(mode)*1000  #for instance: if mode 16, sr = 16kHz
    if mode == '16':
      window_size = sampling_rate
      overlap = int(window_size*0.75)
    elif mode == '22':
      sampling_rate=22050
      window_size = sampling_rate
      overlap = int(window_size*0.5)
    elif mode == '32':
      window_size = sampling_rate
      overlap = int(window_size*0.5)
    
    hop_length = (window_size-overlap)
    return sampling_rate, window_size, hop_length
'''

#in case we need other feature shape
def analysis_parameters(mode):
  sampling_rate=int(mode)*1000  #for instance: if mode 16, sr = 16kHz
  
  if mode=='16':
    window_size = 512
    overlap = int(window_size*0.75)
    hop_length = (window_size-overlap)
  elif mode=='8':
    window_size = 256
    overlap = int(window_size*0.5)
    hop_length = (window_size-overlap)
  return sampling_rate, window_size, hop_length
'''

def preprocess_data(audio_path, df, folds, files, mode):
    
  def _windows(data, window_size, hop_length):
    start = 0
    while start < len(data):
      yield start, start + window_size
      start += hop_length

  def normalize(data):#in the space [-1,1]
    #return (data - np.min(data)) / (np.max(data) - np.min(data))#[0,1]
    return (2*(data-np.min(data))/(np.max(data) - np.min(data)))-1
  
  def flatten_features(feature):
    feature = feature.T.flatten()
    return feature

  features, labels, folders = [], [], []  
 
  extr = True
  if extr == True:
    
    print('Preprocessing data ........ ')
    
    sampling_rate, window_size, hop_length = analysis_parameters(mode)
    print(f'sampling_rate: {sampling_rate}, window_size: {window_size}, hop_length: {hop_length}')

    shape_print=True

    #deterministic random augmentation
    np.random.seed(3)

    for count_files, file in tqdm(enumerate(audiofiles)):
      #label = int(df.loc[df['filename']==file]['target'].to_string(index=False))
      #folder = int(df.loc[df['filename']==file]['fold'].to_string(index=False))
      name = file.split('.wav')[0]
      label = int(file.split('-')[-1].split('.')[0])
      folder = int(file.split('-')[0])

      #readfile
      raw, sr =librosa.load(audio_path+file, sr=sampling_rate, mono=True)
      #print(f'{file} had length {len(raw)}')

      #normalize
      raw = normalize(raw)
      
      frames = []
      #get windows out of the raw waveform
      for count_frames,(start,end) in enumerate(_windows(raw,window_size,hop_length)):
        if(len(raw[start:end]) == window_size):
          #print(start,end)
          frame = raw[start:end]#rectangular window

          #flatten
          #implemented inside the model

          frames.append(frame)
      
      features.append(frames)
      labels.append(label)
      folders.append(folder)

      #Synthetic data augmentations
      '''Probability of a file being augmented is 100% so as to increase to the double the size of the ESC dataset,and to prevent from imbalanced class distribution issues
      #fprob = np.random.uniform(low=0,high=1)
      #if fprob>0.75:
      '''
      category = df.loc[df['filename']==file]['category'].to_string(index=False).lstrip()
      augmented = audio_augmentation(data=raw,sr=sr,class_conditional=category)
      synth_frames = []
      for count_frames,(start,end) in enumerate(_windows(augmented,window_size,hop_length)):
        if(len(augmented[start:end]) == window_size):
          #print('AUGMENTED',start,end)
          synthetic_frame = augmented[start:end]#rectangular window

          synth_frames.append(synthetic_frame)
      
      features.append(synth_frames)
      labels.append(label)
      folders.append(folder)

      if shape_print:
            print('\nFeature Shape Check\n')
            print(f'raw has len:{len(raw)/sampling_rate}')
            print(f'Postprocessed feature has shape : {np.asarray(features).shape} with min:{np.asarray(features[0]).min()} and max:{np.asarray(features[0]).max()}]')
            print()
            shape_print = False

  #'''
  print('len(features)-features',len(features))
  print('len(features[0])-freq_domain',len(features[0]))
  print('labels',len(labels))
  print('folders',len(folders))
  #'''
  return features, labels, folders

In [ ]:
#get analysis parameters
sampling_rate, window_size, hop_length = analysis_parameters(mode)
print(f'sampling_rate: {sampling_rate}, window_size: {window_size}, hop_length: {hop_length}')

#extract_features       
features, labels, folders = preprocess_data(audio_path, esc50, folds, audiofiles, mode)
#features, labels, folders = extract_mel_spectogram(audio_path,us8k,folds,audiofiles,sampling_rate,hop_length,fft_points,mel_bands)
print(f' feature\'s len : {len(features)}, labels : {len(labels)}, folders : {len(folders)}')

sampling_rate: 32000, window_size: 32000, hop_length: 16000
Preprocessing data ........ 
sampling_rate: 32000, window_size: 32000, hop_length: 16000


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:67: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



Feature Shape Check

raw has len:5.0
Postprocessed feature has shape : (2, 9, 32000) with min:-1.0 and max:1.0]


len(features)-features 4000
len(features[0])-freq_domain 9
labels 4000
folders 4000
 feature's len : 4000, labels : 4000, folders : 4000


In [ ]:
#train and fold definition
vfold = 1
train_folds = [folds.index(fold)+1 for i,fold in enumerate(folds) if vfold!=folds.index(fold)+1]
#train_folds = [folds.index(fold)+1 for i,fold in enumerate(folds) if (vfold-1)==folds.index(fold)+1]#for debugging
valid_folds = [folds.index(fold)+1 for i,fold in enumerate(folds) if vfold==folds.index(fold)+1]
print('train_folds: ',train_folds)
print('valid_folds: ',valid_folds)

train_folds:  [2, 3, 4, 5]
valid_folds:  [1]


In [ ]:
class Data(Dataset): 
  def __init__(self, features,labels,folders,split):
    
    def convert_to_tensor(data):
      tensors = []
      for file in data:
        tensor = torch.Tensor(list(file))
        tensors.append(tensor)
      return tensors

    print('Loading_features......')

    #features, labels, folders
    self.indexes = [i for i, val in enumerate(folders) if val in split]
    self.data = [features[x] for x in self.indexes]
    self.labels = [labels[x] for x in self.indexes]

    #convert numpy to tensor
    self.data = convert_to_tensor(self.data)

  def __len__(self):
    return len(self.data)
  def __getitem__(self, idx):#load data on demand
    return self.data[idx], self.labels[idx]

In [ ]:
#load features
train_data = Data(features,labels,folders,train_folds)
valid_data = Data(features,labels,folders,valid_folds)
print('features are loaded')

Loading_features......
Loading_features......
features are loaded


In [ ]:
#define batch size
batch_size = 16
print(f'batch_size: {batch_size}')

#data iterator
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=False)

batch_size: 16


In [ ]:
class R1Dcnn9(nn.Module):
    def __init__(self,num_cats):
        super(R1Dcnn9, self).__init__()

        self.conv1 = nn.Conv1d(1, 32, kernel_size=64, stride=2)#64
        self.bn1 = nn.BatchNorm1d(32)
        self.conv2 = nn.Conv1d(32, 32, kernel_size=64, stride=2)#64
        self.bn2 = nn.BatchNorm1d(32)
        self.pool1 = nn.MaxPool1d(kernel_size=8, stride=4) 
        self.conv3 = nn.Conv1d(32, 64, kernel_size=32, stride=2)
        self.bn3 = nn.BatchNorm1d(64)
        self.conv4 = nn.Conv1d(64, 64, kernel_size=32, stride=2)
        self.bn4 = nn.BatchNorm1d(64)
        self.pool2 = nn.MaxPool1d(kernel_size=8, stride=2) 
        self.conv5 = nn.Conv1d(64, 128, kernel_size=16, stride=2) 
        self.bn5 = nn.BatchNorm1d(128)      
        self.conv6 = nn.Conv1d(128, 128, kernel_size=8, stride=2) 
        self.bn6 = nn.BatchNorm1d(128)
        self.pool3 = nn.MaxPool1d(kernel_size=4, stride=2) 
        self.conv7 = nn.Conv1d(128, 256, kernel_size=4, stride=2) 
        self.bn7 = nn.BatchNorm1d(256)
        self.conv8 = nn.Conv1d(256, 256, kernel_size=4, stride=2) 
        self.bn8 = nn.BatchNorm1d(256)
        self.pool4 = nn.MaxPool1d(kernel_size=2, stride=2) 
        self.drop1 = nn.Dropout(0.25)
        self.relu = nn.ReLU()
        self.ap = nn.AdaptiveAvgPool1d(7)
        self.fc1 = nn.Linear(256*7, 512)
        self.drop2 = nn.Dropout(0.16)       
        self.fc2 = nn.Linear(512, num_cats)
        
    def forward(self, x):

        x = x.view(x.shape[0], 1,-1 )
        x = self.conv1(x)
        x = F.relu(self.bn1(x))
        x = self.conv2(x)
        x = F.relu(self.bn2(x))
        x = self.pool1(x)
        x = self.conv3(x)
        x = F.relu(self.bn3(x))
        x = self.conv4(x)
        x = F.relu(self.bn4(x))
        x = self.pool2(x)
        x = self.conv5(x)
        x = F.relu(self.bn5(x))
        x = self.conv6(x)
        x = F.relu(self.bn6(x))
        x = self.pool3(x)
        x = self.conv7(x)
        x = F.relu(self.bn7(x))
        x = self.conv8(x)
        x = F.relu(self.bn8(x))
        x = self.pool4(x)
        x = self.ap(x)
        x = x.view(x.shape[0], x.size(1) * x.size(2))
        x = self.relu(self.fc1(self.drop1(x)))
        x = self.fc2(self.drop2(x))
        x = F.log_softmax(x, dim = 1)
        return x    

In [ ]:
#introduce reproducibility
seed = 70
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

#init model
model =  R1Dcnn9(num_cats=num_classes).to(device)#.cuda()#
modelid='R1Dcnn9'
total_params = sum(p.numel() for p in model.parameters())
print(f'model_{modelid} initialized with total : {total_params} parameters.')

#define loss and optimizer
loss_fn = nn.CrossEntropyLoss()
learning_rate = 1e-5
epochs = 115
print(f' learning_rate = {learning_rate}, epochs = {epochs}')
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

model_R1Dcnn9 initialized with total : 1866098 parameters.
 learning_rate = 1e-05, epochs = 115


In [ ]:
#define metrics
def F1_score(trace_y, trace_yhat, classes):
  num_classes = len(classes)
  #confussion matrix
  confmat = confusion_matrix(trace_y, trace_yhat, num_classes)
    
  TP=pd.Series(confmat.to_numpy().diagonal())
  FP=confmat.sum(axis=0)-TP
  FN=confmat.sum(axis=1)-TP

  TN=confmat.sum().sum()-TP-FP-FN
  
  #micro
  micro_precision = TP.sum()/(TP.sum()+FP.sum())
  micro_recall = TP.sum()/(TP.sum()+FN.sum())
  F1_micro=2*micro_precision*micro_recall/(micro_precision+micro_recall)

  #macro
  macro_precision = pd.Series(np.nan)
  macro_recall= pd.Series(np.nan)
  macro_f1 = pd.Series(np.nan)
  #Avoid Zero-Division
  for i in range(num_classes):
    if (TP[i]==0 and FP[i]==0):
      macro_precision[i]=0
    else:
      macro_precision[i] = TP[i]/(TP[i]+FP[i]) 
    if (TP[i]==0 and FN[i]==0):
      macro_recall[i]=0
    else:
      macro_recall[i] = TP[i]/(TP[i]+FN[i])
    
    if (macro_precision[i]+macro_recall[i]==0.0):
      macro_f1[i]=0
    else:
      macro_f1[i] = 2*macro_precision[i]*macro_recall[i]/(macro_precision[i]+macro_recall[i])
      
  macro_precision = macro_precision.sum()/num_classes
  macro_recall=macro_recall.sum()/num_classes
  F1_macro = macro_f1.sum()/num_classes

  return micro_recall,micro_precision,F1_micro, macro_recall,macro_precision,F1_macro
  
#labels, preds
def confusion_matrix(trace_y,trace_yhat, num_classes):
  confmat=pd.DataFrame(data=np.zeros(shape=(num_classes,num_classes)))
  predictions=trace_yhat.argmax(axis=1)

  for i,pred in enumerate(predictions):
    confmat.iat[trace_y[i],pred]+=1

  return confmat


def F1_Class(trace_y,trace_yhat,classes):
  num_classes = len(classes)
  confmat = confusion_matrix(trace_y, trace_yhat, num_classes)
  
  TP=pd.Series(confmat.to_numpy().diagonal())
  FP=confmat.sum(axis=0)-TP
  FN=confmat.sum(axis=1)-TP
  TN=confmat.sum().sum()-TP-FP-FN

  #AVOID ZERO DIVISION
  class_precision = pd.Series(np.nan)
  class_recall = pd.Series(np.nan)
  class_f1 = pd.Series(np.nan)
  for i in range(num_classes):
    if (TP[i]==0 and FP[i]==0):
      class_precision[i]=0
    else:
      class_precision[i] = TP[i]/(TP[i]+FP[i]) 
    if (TP[i]==0 and FN[i]==0):
      class_recall[i]=0
    else:
      class_recall[i] = TP[i]/(TP[i]+FN[i])
    
    if (class_precision[i]==0.0 and class_recall[i]==0.0):
      class_f1[i]=0
    else:
      class_f1[i] = 2*class_precision[i]*class_recall[i]/(class_precision[i]+class_recall[i])
    
  #create a dict_report
  f1_class_report = {}
  class_report = {}
  class_counts = np.asarray(np.unique(trace_y, return_counts=True)).T

  for i,c in enumerate(classes):
    f1_class_report[c] = {}
    f1_class_report[c]['precision'] = class_precision[i]
    f1_class_report[c]['recall'] = class_recall[i]
    f1_class_report[c]['f1'] = class_f1[i]
    f1_class_report[c]['count'] = class_counts[i][1]

  return f1_class_report


from sklearn.metrics import classification_report
import sklearn
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support

def backup_metrics(labels,preds,classes,path,vfold):
    if (not os.path.exists(path)):
      os.makedirs(path)
    report = classification_report(labels, preds.argmax(1), target_names=classes)
    report_data = []
    lines = report.split('\n')
    for line in lines[2:-5]:
        row = {}
        row_data = ' '.join(line.split())   
        row_data = row_data.split(' ')
        row['class'] = row_data[0]
        row['precision'] = float(row_data[1])
        row['recall'] = float(row_data[2])
        row['f1_score'] = float(row_data[3])
        row['support'] = float(row_data[4])
        report_data.append(row)
    dataframe = pd.DataFrame.from_dict(report_data)
    dataframe.to_csv(path+'backup_classification_report_'+str(vfold)+'.csv',index=False)

    accuracy= f1_score(labels, preds.argmax(1), average='micro', zero_division='warn')
    macro_avg= f1_score(labels, preds.argmax(1), average='macro', zero_division='warn')
    weightedf1= f1_score(labels, preds.argmax(1), average='weighted', zero_division='warn')

    precision,recall,_,_ =precision_recall_fscore_support(labels, preds.argmax(1), average='macro')
    general = pd.DataFrame(data = [accuracy,recall,precision,macro_avg,weightedf1],index=['accuracy','recall','precision','macro_f1','weightedf1'])
    general.to_csv(path+'backup_general_'+str(vfold)+'.csv')

In [ ]:
#prevent overfitting custom method
class prevent_overfitting:
  def __init__(self):
    self.tolerance=7  #how many epochs tolerance
    self.minloss = 10 
    self.lossenvelope =[]
    self.thresshold = 0.4
    self.avgperformances=[]
    self.best_epoch = 0
    self.micro_accuracies = []
    self.macro_accuracies = []
    self.best_curr_model = False#None #model.state_dict save
  
  #check overfitting by observing 3 last epoch's valid loss
  def detect_overfitting(self,validloss,epoch,console_path):
    self.lossenvelope.append(validloss)
    if validloss<self.minloss:
      self.minloss=validloss
    print('minloss',self.minloss)
    print(f'minloss : {self.minloss}', file=open(console_path, "a"))

    if (len(self.lossenvelope)>self.tolerance):
      self.lossenvelope.pop(0) #reject first value when {tolerance} values are passed

    if(len(self.lossenvelope)>=self.tolerance):
      overfit=all(earlier <= later for earlier, later in zip(self.lossenvelope, self.lossenvelope[-5:]))#check last 5 values if ascending
      if (overfit and min(self.lossenvelope)>self.minloss):
        return True
    return False
  
  #store best model's results by observing mean micro and macro accuracies
  def store_best_model(self,micro_accuracy,macro_accuracy,console_path):
    self.macro_accuracies.append(micro_accuracy)
    self.micro_accuracies.append(macro_accuracy)
    
    meanaccuracy = (micro_accuracy+macro_accuracy)/2.0
    self.avgperformances.append(meanaccuracy)
    
    self.best_epoch = self.avgperformances.index(max(self.avgperformances))+1
    
    if meanaccuracy >= self.thresshold:
      self.best_curr_model = (meanaccuracy >= max(self.avgperformances))
      print(f'just saved the best current model in epoch{self.best_epoch}, with acc1:{self.micro_accuracies[self.best_epoch-1]}, and acc2:{self.macro_accuracies[self.best_epoch-1]}')
      print(f'just saved the best current model in epoch{self.best_epoch}, with acc1:{self.micro_accuracies[self.best_epoch-1]}, and acc2:{self.macro_accuracies[self.best_epoch-1]}', file=open(console_path, "a"))
    else:
      self.best_curr_model = False

    return self.best_curr_model, self.micro_accuracies[self.best_epoch-1],self.macro_accuracies[self.best_epoch-1]#, self.best_epoch
  
  def early_stopping(self,epochinst):
    print('training is terminating so as to prevent further overfitting')
    total_epochs = epochinst.set_total(epochinst.get_step())
    return total_epochs

class epochs_class:
  def __init__(self):
    self.total_epochs=50#random value
    self.step_epoch=1
  def set_total(self,num_epochs):
    self.total_epochs=num_epochs
    return self.get_total()
  def get_total(self):
    return self.total_epochs
  def next_step(self):
    self.step_epoch+=1
    return self.step_epoch
  def get_step(self):
    return self.step_epoch

In [ ]:
#save results
def save_results(content, name):
  path = results_path
  if (not os.path.exists(path)):
    os.mkdir(path)
  ovr_results, class_results = content
  ovr_results_filename, class_results_filename = name 
  
  ovr_results.to_csv(path+ovr_results_filename) 
  class_results.to_csv(path + class_results_filename) 
   
#func to define the last argument in the save_results method
#PATTERN:dataset&exp_mode_validationfold_attempt_metrics.csv
def define_filenames_pattern(expid, mode, vfold, attempt):
  classf1_report_filename = expid+'_'+mode+'_v'+str(vfold)+'_a'+str(attempt)+'_classF1.csv'
  ovr_results_filename=expid+'_'+mode+'_v'+str(vfold)+'_a'+str(attempt)+'_overalF1.csv'
  return ovr_results_filename , classf1_report_filename
  
  
#as the first argument in save_results
def define_content(epochs,
		               mean_train_losses, 
                   mean_valid_losses, 
                   microrecall,
                   microprecision,
                   microf1, 
                   macrorecall,
                   macroprecision,
                   macrof1, 
		               classf1):
                   
  #data = [mean_train_losses, mean_valid_losses, accuracies,micro_auroc, macro_auroc microf1, macrof1]#auroc,
  data = {'mean_train_loss' : mean_train_losses, 'mean_valid_loss' : mean_valid_losses, 
  'micro_recall':microrecall, 'micro_precision': microprecision, 'micro_f1' : microf1,
  'macro_recall':macrorecall, 'macro_precision': macroprecision, 'macro_f1' : macrof1}
  
  #index names
  epochs_index= (['epoch_'+str(ep+1) for ep in range(epochs)])
  
  overal_results = pd.DataFrame(data=data, index=epochs_index,  dtype=np.float16)
  class_results = pd.DataFrame(data=classf1, index=epochs_index,  dtype=np.float16)
  
  overal_results.index.name = 'epochs'
  class_results.index.name = 'epochs'
  
  return overal_results, class_results

 
#save general results so as to compare with other systems in a different folder
def save_genres(micro, macro, params, validation_fold, filename):
  
  path = compare_results_path

  #if csv exists, overwrite results
  if os.path.isfile(path+filename):
    general_results=pd.read_csv(path+filename)
    general_results.set_index('validation_fold:',inplace=True)
    #debug
    print(general_results.shape)
    new_fold_results = [micro, macro, params]
    #debug
    print(len(new_fold_results))
    general_results[validation_fold] = new_fold_results
    general_results.to_csv(path+filename)
  #if not, store them into a new csv file
  else:
    os.makedirs(path)
    data = {validation_fold : [micro,macro,params]}
    general_results = pd.DataFrame(data=data, index=['micro_f1','macro_f1','params'], dtype=np.float16)
    general_results.index.name = 'validation_fold:'
    general_results.to_csv(path+filename) 
    print(general_results)


def genres_filename(expid,mode):
  filename=expid+'_'+str(mode)+'.csv'
  return filename

#save the model
#PATTERN:exp-num_mode_dataset_'model'_attempt		
def save_model(model_name, vfold, state_dict):
  path = model_savepath
  if not os.path.exists(path):
    os.mkdir(path)
  torch.save(state_dict, path + model_name+'_'+str(vfold)+'.pt')
'''
#load model
def load_model(model_name, mode):
  if (mode=='80'):
    model = esc_mel_model_hybrid(input_shape=(1,80,431), batch_size=16, num_cats=50)
  elif(mode=='128'):
    model = esc_mel_model_hybrid(input_shape=(1,128,431), batch_size=16, num_cats=50)
  model.load_state_dict(torch.load(paths.model_savepath+model_name+'.pth'))
'''

In [ ]:
def ZipAnDownload(mode):
  if mode == '16':
    !zip -r /content/esc116.zip /content/esc116
    filez.download("/content/esc116.zip")
  elif mode == '22':
    !zip -r /content/esc122.zip /content/esc122
    filez.download("/content/esc122.zip")
  elif mode == '32':
    !zip -r /content/esc132.zip /content/esc132
    filez.download("/content/esc132.zip")

In [ ]:
class attempt_class:
  #init_attempt
  def __init__(self,mode,vfold):

    self.attempt_path = attempt_path
    self.attempt_file = attempt_path+"attempts_"+mode+"_"+str(vfold)+".txt"

    if (not os.path.exists(self.attempt_path)):
      os.makedirs(self.attempt_path)
      self.init_files()
    elif (not os.path.isfile(self.attempt_file)):
      self.init_files()
    else:
      print(f'attempt is already initialized')
      print(f'Experiment\'s _{mode} attempt no_ : {self.get_attempt()}')
  
  def init_files(self):
    attempt = 1
    f = open(self.attempt_file,'w')
    with open(self.attempt_file, 'a') as out:
      out.write(str(attempt))
      print(f'Experiment\'s attempt no_ : {attempt}')
  
  def add_attempt(self):
    with open(self.attempt_file, "r") as f:
      attempt = f.read()
      attempt = int(attempt)
      attempt+=1
    with open(self.attempt_file, "w") as f:
      f.write(str(attempt))
    print(f'Experiment\'s attempt changed to : {attempt}')

  def get_attempt(self):
    with open(self.attempt_file, "r") as f:
      attempt = int(f.read())
    return attempt

  def set_attempt(self,attempt_value):
    with open(self.attempt_file, "w") as f:
      f.write(str(attempt_value))
    print(f'Experiment\'s attempt is set to : {attempt_value}')

In [ ]:
#train_model_func
def train(model, loss_fn, train_loader, valid_loader,
          epochs, optimizer, learning_rate, device, classes, expid, mode, vfold, modelid):

  exp_attempt = attempt_class(mode, vfold)
  
  print('Train started..')

  train_losses = []
  valid_losses = []
  
  microrecall = []
  macrorecall = []
  
  microprecision = []
  macroprecision = []
  
  microf1 = []
  macrof1 = []
  
  classf1 = []
  
  #console print redirect - save output of train to a log_file
  console_path = "/content/"+expid+mode+'/'+expid+'_'+str(mode)+'_'+str(vfold)+".txt"
  
  epoch_instance = epochs_class()
  total_epochs = epoch_instance.set_total(epochs)
  epoch = epoch_instance.get_step()

  prevent_overfit = prevent_overfitting()
  
  model_name = expid+'_'+mode+'_v'+str(vfold)+'_a'+str(exp_attempt.get_attempt())+'_'+modelid
  
  #Train step
  while (epoch<=total_epochs):

    model.train()
    batch_losses=[]

    for i,data in tqdm(enumerate(train_loader)):
      x, y = data
      optimizer.zero_grad()
      x = x.to(device, dtype=torch.float32)
      y = y.to(device, dtype=torch.long) 
      y_hat = model(x) 
      
      
      loss = loss_fn(y_hat, y)
      loss.backward()
      
      batch_losses.append(loss.item())						
      optimizer.step()
    train_losses.append(batch_losses)
    mean_train_losses=([np.mean(l) for l in train_losses])
    
    print()
    print(f'Epoch - {epoch} Train-Loss : {np.mean(train_losses[-1])}')
    print(f'\nEpoch - {epoch} Train-Loss : {np.mean(train_losses[-1])}\n', file=open(console_path, "a"))
    print()
    
    #Validation step
    model.eval()
     
    batch_losses=[]
    trace_y = []
    trace_yhat = []
    
    for i, data in enumerate(valid_loader):
      x, y = data
      x = x.to(device, dtype=torch.float32)
      y = y.to(device, dtype=torch.long)
      y_hat = model(x)
      loss = loss_fn(y_hat, y)
      trace_y.append(y.cpu().detach().numpy())
      trace_yhat.append(y_hat.cpu().detach().numpy())      
      batch_losses.append(loss.item())
    valid_losses.append(batch_losses)
    mean_valid_losses=([np.mean(l) for l in valid_losses])
    
    trace_y = np.concatenate(trace_y)
    trace_yhat = np.concatenate(trace_yhat)

    #f1,micro,macro
    micro_recall,micro_precision,micro, macro_recall,macro_precision,macro = F1_score(trace_y,trace_yhat,classes)
    
    microrecall.append(micro_recall)
    microprecision.append(micro_precision)
    microf1.append(micro) 

    macrorecall.append(macro_recall)
    macroprecision.append(macro_precision)
    macrof1.append(macro)
    
    #f1 for each class
    f1_class = F1_Class(trace_y,trace_yhat,classes)
    classf1.append(f1_class)

    #console prints
    print(f'Epoch - {epoch} Valid-Loss : {np.mean(valid_losses[-1])}')
    print(f'micro_f1_{epoch} = {micro} ')
    print(f'macro_f1_{epoch} = {macro} ')
    #append log_file
    print(f'Epoch - {epoch} Valid-Loss : {np.mean(valid_losses[-1])}', file=open(console_path, "a"))
    print(f'micro_f1_{epoch} = {micro} ', file=open(console_path, "a"))
    print(f'macro_f1_{epoch} = {macro} ', file=open(console_path, "a"))
    '''
    #check if overfitting and reduce lr twice, after that, in case loss is ascending terminate train    
    early_stop, optimizer = overfit.detect_overfitting(np.mean(valid_losses[-1]),optimizer, learning_rate, epoch_instance)
    if early_stop:
      total_epochs=epoch_instance.set_total(epoch_instance.get_step())
    epoch=epoch_instance.next_step()
    '''
    #check if overfitting and reduce lr twice, after that, in case loss is ascending terminate train    
    early_stop = prevent_overfit.detect_overfitting(np.mean(valid_losses[-1]), epoch, console_path)
    if early_stop:
      total_epochs=prevent_overfit.early_stopping(epoch_instance)
    
    #always store the best according to avg_micro_and_macro_F1
    current_best, best_micro, best_macro = prevent_overfit.store_best_model(micro,macro,console_path)#, best_epoch
    #If achieves current best mean accuracy, Save the model
    if current_best:
      save_model(model_name, vfold, model.state_dict())
      backup_metrics(trace_y,trace_yhat,classes,results_path+'backup/')
    
    #next epoch
    epoch=epoch_instance.next_step()
  
  #calc_trainable_params
  #params=sum(p.numel() for p in model.parameters() if p.requires_grad)
  model_parameters = filter(lambda p: p.requires_grad, model.parameters())
  params = sum([np.prod(p.size()) for p in model_parameters])
      
  #Save analytic results 
  save_results(define_content(total_epochs,
		                                     mean_train_losses,
                                         mean_valid_losses, 
                                         microrecall,
                                         microprecision,
                                         microf1, 
                                         macrorecall,
                                         macroprecision,
                                         macrof1,
		                                     classf1),
                      define_filenames_pattern(expid, mode, vfold, exp_attempt.get_attempt()))
  
      
  #Save general results so as to quickly compare systems
  genresfilename = genres_filename(expid,mode)
  save_genres(round(best_micro,3), round(best_macro,3), params, vfold, genresfilename)
  print('params =', params)
  print('params =', params, file=open(console_path, "a"))

  exp_attempt.add_attempt()
  ZipAnDownload(mode)  
  return  train_losses,valid_losses

In [ ]:
#train single-fold
train_losses,valid_losses = train(model, loss_fn, train_loader, valid_loader, epochs, optimizer, learning_rate, device, esc_classes, expid, mode, vfold, modelid)

In [ ]:
if mode =='8':
  !rm -r esc18 esc18.zip
if mode =='16':
  !rm -r esc116 esc116.zip
if mode =='22':
  !rm -r esc122 esc122.zip
if mode =='32':
  !rm -r esc132 esc132.zip